In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 18:49:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_taxi = spark.read \
    .option("header", "true") \
    .option("inferSchema",True)\
    .csv('taxi+_zone_lookup.csv')

In [4]:
df_fhv = spark.read \
    .option("header", "true") \
    .option("inferSchema",True)\
    .csv('fhvhv_tripdata_2021-06.csv.gz')

In [6]:
df_fhv.rdd.getNumPartitions()

1

In [8]:
df_fhv = df_fhv.repartition(12)

In [10]:
df_fhv.write.parquet("fhvhv_06_21")

23/03/04 18:52:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [11]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [16]:
from pyspark.sql.functions import month,col,dayofmonth
df_fhv.filter(month(col("pickup_datetime")) == 6).filter(dayofmonth(col("pickup_datetime")) == 15).count()

452470

In [17]:
df_fhv = df_fhv.withColumn("time_diff_seconds", col("dropoff_datetime").cast("long") - col("pickup_datetime").cast("long"))
df_fhv = df_fhv.withColumn("time_diff_hours", col("time_diff_seconds")/3600)

In [18]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)
 |-- time_diff_seconds: long (nullable = true)
 |-- time_diff_hours: double (nullable = true)



In [21]:
from pyspark.sql.functions import max
df_fhv.select(max("time_diff_hours")).show()

+--------------------+
|max(time_diff_hours)|
+--------------------+
|    66.8788888888889|
+--------------------+



In [22]:
df_taxi.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [23]:
df_fhv_joined = df_fhv.join(
    df_taxi,
    on=df_taxi.LocationID == df_fhv.PULocationID,
    how='left'
)

In [25]:
df_fhv_joined.groupby("Zone").count().orderBy("count",ascending=False).take(1)

[Row(Zone='Crown Heights North', count=231279)]